# Smart Parking API - Sample Data Generator

This notebook demonstrates how to generate sample data for the Smart Parking API. The API expects vehicle data in a specific format for parking allocation.

## API Schema Structure

The main request body follows this schema:
```json
{
  "vehicles": [
    {
      "vehicle_plate_num": "string",
      "vehicle_plate_type": 0,
      "vehicle_type": 0,
      "arrival_time": "string",
      "departure_time": "string",
      "priority_level": 0
    }
  ],
  "allocation_strategy": "string"
}
```

In [ ]:
# Import Required Libraries
import json
import random
from datetime import datetime, timedelta
from typing import Dict, List, Any
import pandas as pd

# Set random seed for reproducible results
random.seed(42)

print("Libraries imported successfully!")

## Vehicle Data Field Descriptions

Based on the codebase analysis, here are the field definitions:

| Field | Type | Values | Description |
|-------|------|--------|-------------|
| `vehicle_plate_num` | string | Any string | Unique vehicle identifier (e.g., "ABC1234") |
| `vehicle_plate_type` | int | 0, 1, 2 | 0: Private, 1: Public, 2: Government |
| `vehicle_type` | int | 0, 1, 2 | 0: Car, 1: Truck, 2: Motorcycle |
| `arrival_time` | string | ISO datetime | When vehicle arrives (e.g., "2025-06-26T08:30:00") |
| `departure_time` | string | ISO datetime | When vehicle leaves (e.g., "2025-06-26T12:30:00") |
| `priority_level` | int | 0-3 | Priority level (0: Lowest, 3: Highest) |

## Allocation Strategy Options

- `"sequential"` - Allocate slots in sequential order
- `"random"` - Randomly assign available slots  
- `"algorithm"` - Use AI/ML algorithm for optimal allocation

In [ ]:
# Define Vehicle Data Schema and Helper Functions

def generate_vehicle_plate():
    """Generate a realistic vehicle plate number"""
    prefixes = ['ABC', 'XYZ', 'PQR', 'LMN', 'GOV', 'PUB']
    numbers = random.randint(1000, 9999)
    return f"{random.choice(prefixes)}{numbers}"

def generate_arrival_time(base_date=None):
    """Generate realistic arrival time with peak hour bias"""
    if base_date is None:
        base_date = datetime.now().replace(hour=0, minute=0, second=0, microsecond=0)
    
    # Weight towards peak hours (8-10am, 5-7pm)
    if random.random() < 0.6:  # 60% chance of peak hours
        if random.choice([True, False]):  # Morning or evening peak
            hour = random.choice([8, 9, 10])  # Morning peak
        else:
            hour = random.choice([17, 18, 19])  # Evening peak
    else:
        hour = random.randint(6, 21)  # Regular hours
    
    minute = random.randint(0, 59)
    arrival = base_date + timedelta(hours=hour, minutes=minute)
    return arrival.isoformat()

def generate_departure_time(arrival_time_str, vehicle_type):
    """Generate departure time based on arrival and vehicle type"""
    arrival = datetime.fromisoformat(arrival_time_str)
    
    # Duration based on vehicle type (from codebase analysis)
    if vehicle_type == 0:  # Car
        duration_hours = random.choices([1, 2, 3, 4, 5, 8], weights=[0.2, 0.3, 0.2, 0.15, 0.1, 0.05])[0]
    elif vehicle_type == 1:  # Truck
        duration_hours = random.choices([1, 2, 3, 4, 6], weights=[0.1, 0.2, 0.3, 0.25, 0.15])[0]
    else:  # Motorcycle
        duration_hours = random.choices([1, 2, 3, 4], weights=[0.4, 0.3, 0.2, 0.1])[0]
    
    departure = arrival + timedelta(hours=duration_hours)
    return departure.isoformat()

def calculate_priority_level(plate_type, vehicle_type):
    """Calculate priority based on plate and vehicle type (from codebase logic)"""
    base_priority = 0
    
    # Vehicle plate type priority
    if plate_type == 2:  # Government
        base_priority += 2
    elif plate_type == 1:  # Public
        base_priority += 1
    
    # Vehicle type adjustments
    if vehicle_type == 1:  # Truck - lower priority due to size
        base_priority = max(0, base_priority - 1)
    
    # Cap priority between 0-3
    return min(3, base_priority)

print("Helper functions defined successfully!")

In [ ]:
# Generate Sample Vehicle Data

def generate_single_vehicle_sample():
    """Generate one realistic vehicle data sample"""
    
    # Generate basic vehicle info
    plate_type = random.choice([0, 1, 2])  # Private, Public, Govt
    vehicle_type = random.choice([0, 1, 2])  # Car, Truck, Motorcycle
    
    # Generate times
    arrival_time = generate_arrival_time()
    departure_time = generate_departure_time(arrival_time, vehicle_type)
    
    # Calculate priority
    priority_level = calculate_priority_level(plate_type, vehicle_type)
    
    vehicle_data = {
        "vehicle_plate_num": generate_vehicle_plate(),
        "vehicle_plate_type": plate_type,
        "vehicle_type": vehicle_type,
        "arrival_time": arrival_time,
        "departure_time": departure_time,
        "priority_level": priority_level
    }
    
    return vehicle_data

# Generate multiple sample vehicles
sample_vehicles = []
for i in range(5):
    sample_vehicles.append(generate_single_vehicle_sample())

print(f"Generated {len(sample_vehicles)} sample vehicles")
print("\nFirst sample vehicle:")
print(json.dumps(sample_vehicles[0], indent=2))

## Complete API Request Examples

Here are complete sample requests for different scenarios:

In [ ]:
# Create Complete API Request Examples

# Example 1: Single Vehicle - Government Car (High Priority)
government_car_request = {
    "vehicles": [
        {
            "vehicle_plate_num": "GOV2024",
            "vehicle_plate_type": 2,  # Government
            "vehicle_type": 0,       # Car
            "arrival_time": "2025-06-26T08:30:00",
            "departure_time": "2025-06-26T10:30:00",
            "priority_level": 2       # High priority
        }
    ],
    "allocation_strategy": "algorithm"
}

# Example 2: Multiple Vehicles - Mixed Types
multiple_vehicles_request = {
    "vehicles": [
        {
            "vehicle_plate_num": "ABC1234",
            "vehicle_plate_type": 0,  # Private
            "vehicle_type": 0,       # Car
            "arrival_time": "2025-06-26T09:15:00",
            "departure_time": "2025-06-26T13:15:00",
            "priority_level": 0       # Low priority
        },
        {
            "vehicle_plate_num": "PUB5678",
            "vehicle_plate_type": 1,  # Public
            "vehicle_type": 1,       # Truck
            "arrival_time": "2025-06-26T10:00:00",
            "departure_time": "2025-06-26T14:00:00",
            "priority_level": 0       # Adjusted down for truck
        },
        {
            "vehicle_plate_num": "XYZ9999",
            "vehicle_plate_type": 0,  # Private
            "vehicle_type": 2,       # Motorcycle
            "arrival_time": "2025-06-26T11:30:00",
            "departure_time": "2025-06-26T13:30:00",
            "priority_level": 0       # Low priority
        }
    ],
    "allocation_strategy": "random"
}

# Example 3: Peak Hour Scenario
peak_hour_request = {
    "vehicles": [
        {
            "vehicle_plate_num": "RUSH001",
            "vehicle_plate_type": 1,  # Public
            "vehicle_type": 0,       # Car
            "arrival_time": "2025-06-26T17:30:00",  # Evening peak
            "departure_time": "2025-06-26T19:30:00",
            "priority_level": 1       # Medium priority
        }
    ],
    "allocation_strategy": "sequential"
}

print("✅ Created sample API requests for different scenarios!")

In [ ]:
# Display Generated Sample Data

print("=" * 80)
print("SMART PARKING API - SAMPLE DATA EXAMPLES")
print("=" * 80)

print("\n📋 Example 1: Single Government Vehicle (High Priority)")
print("-" * 60)
print(json.dumps(government_car_request, indent=2))

print("\n📋 Example 2: Multiple Mixed Vehicles")
print("-" * 60)
print(json.dumps(multiple_vehicles_request, indent=2))

print("\n📋 Example 3: Peak Hour Scenario")
print("-" * 60)
print(json.dumps(peak_hour_request, indent=2))

print("\n📋 Example 4: Random Generated Vehicles")
print("-" * 60)
random_request = {
    "vehicles": sample_vehicles[:3],  # Use first 3 generated vehicles
    "allocation_strategy": "algorithm"
}
print(json.dumps(random_request, indent=2))

## Real Examples from Database

Based on the actual database content in the codebase, here are real allocation examples:

In [ ]:
# Real Database Examples (from parking_db.json)

# These are actual allocations found in the codebase database
real_database_examples = [
    {
        "id": 1,
        "vehicle_plate_num": "ABC123",
        "vehicle_plate_type": 1,  # Public
        "vehicle_type": 2,       # Motorcycle
        "bay_assigned": 1,
        "slot_assigned": 1,
        "allocation_score": 94.06,
        "allocation_time": "2025-04-28T10:30:00Z",
        "departure_time": "2025-04-28T16:45:00Z",
        "priority_level": 0,
        "is_active": True
    },
    {
        "id": 2,
        "vehicle_plate_num": "ABC1234",
        "vehicle_plate_type": 2,  # Government
        "vehicle_type": 1,       # Truck
        "bay_assigned": 3,
        "slot_assigned": 2,
        "allocation_score": 173.21,
        "allocation_time": "2025-04-28T09:30:00+00:00",
        "departure_time": "2025-04-28T11:30:00+00:00",
        "priority_level": 1,
        "is_active": True
    },
    {
        "id": 3,
        "vehicle_plate_num": "ACZ22003",
        "vehicle_plate_type": 2,  # Government
        "vehicle_type": 0,       # Car
        "bay_assigned": 1,
        "slot_assigned": 1,
        "allocation_score": 166.83,
        "allocation_time": "2025-05-20T06:32:00.380Z",
        "departure_time": "2025-05-20T07:32:00.380Z",
        "priority_level": 2,
        "is_active": True
    }
]

print("📊 Real Database Examples:")
print("=" * 50)
for example in real_database_examples:
    print(f"Vehicle: {example['vehicle_plate_num']}")
    print(f"Type: {'Private' if example['vehicle_plate_type']==0 else 'Public' if example['vehicle_plate_type']==1 else 'Government'}")
    print(f"Vehicle: {'Car' if example['vehicle_type']==0 else 'Truck' if example['vehicle_type']==1 else 'Motorcycle'}")
    print(f"Priority: {example['priority_level']}")
    print(f"Assigned: Bay {example['bay_assigned']}, Slot {example['slot_assigned']}")
    print(f"Score: {example['allocation_score']:.2f}")
    print(f"Duration: {example['allocation_time']} → {example['departure_time']}")
    print("-" * 30)

## How to Use These Samples

### API Endpoints

1. **Single Allocation**: `POST /api/parking/allocate`
   - Send one vehicle object from the `vehicles` array
   - Don't include the `allocation_strategy` field

2. **Simulation**: `POST /api/parking/simulate`
   - Send the complete request with `vehicles` array and `allocation_strategy`

3. **Get Status**: `GET /api/parking/status`
   - No request body needed

### Copy-Paste Ready Examples

You can copy any of the JSON examples above and use them directly with your API client (Postman, curl, etc.).

In [ ]:
# Summary Table of All Possible Combinations

import pandas as pd

# Create a comprehensive lookup table
plate_types = {0: "Private", 1: "Public", 2: "Government"}
vehicle_types = {0: "Car", 1: "Truck", 2: "Motorcycle"}

combinations = []
for plate_type_num, plate_type_name in plate_types.items():
    for vehicle_type_num, vehicle_type_name in vehicle_types.items():
        priority = calculate_priority_level(plate_type_num, vehicle_type_num)
        combinations.append({
            "Plate Type Code": plate_type_num,
            "Plate Type": plate_type_name,
            "Vehicle Type Code": vehicle_type_num,
            "Vehicle Type": vehicle_type_name,
            "Calculated Priority": priority,
            "Priority Level": f"{'Low' if priority == 0 else 'Medium' if priority == 1 else 'High' if priority == 2 else 'Highest'}"
        })

df = pd.DataFrame(combinations)
print("📊 Complete Vehicle Type & Priority Matrix:")
print("=" * 70)
print(df.to_string(index=False))

print("\n🎯 Quick Reference for JSON:")
print("• vehicle_plate_type: 0=Private, 1=Public, 2=Government")
print("• vehicle_type: 0=Car, 1=Truck, 2=Motorcycle")
print("• priority_level: 0=Lowest, 1=Low, 2=High, 3=Highest")
print("• allocation_strategy: 'sequential', 'random', 'algorithm'")